<a href="https://colab.research.google.com/github/PayamAkto/GEXERGY/blob/main/CNN_Exercise_TF_Templogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Create synthetic temperature log data with three main scenarios

## 1.1 load function for creating three temperature log scenarios

In [1]:
# import packages
import numpy as np
from scipy.ndimage.filters import gaussian_filter

def CreateTrainingData5(n_m, n_x):
    
    MoveGaussian = np.arange(1, 20)
    startT = 20   
    maxTargetTemperature = 200
    minTargeTemperature = 120
    TargetTemperatures = np.round(minTargeTemperature+np.random.rand(n_m,1)*(maxTargetTemperature-minTargeTemperature))


    maxPerturbLength = 0.50*n_x
    minPerturbLength = 0.12*n_x
    PerturbLengths = np.round(minPerturbLength+np.random.rand(n_m,1)*(maxPerturbLength-minPerturbLength))
       
    maxPerturbPoint = 0.9*n_x
    minPerturbPoint = 0.3*n_x
    startPerturbPoints = np.round(minPerturbPoint+np.random.rand(n_m,1)*(maxPerturbPoint-minPerturbPoint))
    EndPerturbPoints = np.zeros((n_m,1))
    
    #pressure stuff
    startP = 0.1 #MPa   
    maxTargetP = 16
    minTargeP = 20
    TargetPressures = np.round(minTargeP+np.random.rand(n_m,1)*(maxTargetP-minTargeP))

    TemperatureCurves = np.zeros([n_m, n_x])
    PressureCurves = np.zeros([n_m, n_x])
    ReservoirPressure = np.zeros([n_m, 1])
    ReservoirPressureDepth = np.zeros([n_m, 1])
    ReservoirTemperature = np.zeros([n_m, 1])
    
    for m in range(n_m):
        s = int(startPerturbPoints[m])
        e = int(startPerturbPoints[m]+PerturbLengths[m])
        if(e>=n_x):
            e=n_x-1
        GeoGradLine = np.linspace(startT, TargetTemperatures[m], num=n_x)

        StartT = GeoGradLine[s].copy() 
        IsoT = GeoGradLine[e].copy()
       
        randomFloat = np.random.rand()
        if(randomFloat<0.33):#situation number 2
            GeoGradLine[s:e] = GeoGradLine[e].copy()
            startPtIdx_1 = s
            endPtIdx_1 = int(s+np.random.uniform(0, 0.15)*(e-s))+1
            m_slope_1 = (IsoT-StartT)/(endPtIdx_1-startPtIdx_1)
            b_1 = IsoT - m_slope_1*endPtIdx_1

            startPtIdx_2 = s
            endPtIdx_2 = int(s+np.random.uniform(0.3, 0.7)*(e-s))
            startPtT_2 = StartT + np.random.uniform(0.5, 0.85)*(IsoT-StartT)
            m_slope_2 = (IsoT-startPtT_2)/(endPtIdx_2-startPtIdx_2)
            b_2 = IsoT - m_slope_2*endPtIdx_2

            x2 = np.arange(startPtIdx_2,endPtIdx_2)
            l2 = (m_slope_2*x2+b_2).reshape(len(x2),1) 
            l1 = (m_slope_1*x2+b_1).reshape(len(x2),1) 
            x = np.min(np.hstack((l1, l2)), axis=1)
            GeoGradLine[startPtIdx_2:endPtIdx_2] = x.reshape(-1,1)              
            sigma = np.random.randint(1,3)
            TemperatureCurves[m, :] = gaussian_filter(GeoGradLine[:,0], sigma=sigma)
            startPerturbPoints[m] = int((b_2-b_1)/(m_slope_1-m_slope_2)+MoveGaussian[sigma])
            if(e>=n_x-2):
                 EndPerturbPoints[m] = e
            else:
                EndPerturbPoints[m] = e-int(MoveGaussian[sigma]/2)
        elif(randomFloat<0.66): #situation number 1
            GeoGradLine[s:e] = GeoGradLine[e].copy()
            startPtIdx_1 = s
            endPtIdx_1 = int(s+np.random.uniform(0.1, 0.5)*(e-s))
            m_slope_1 = (IsoT-StartT)/(endPtIdx_1-startPtIdx_1)
            b_1 = IsoT - m_slope_1*endPtIdx_1
            x = np.arange(startPtIdx_1,endPtIdx_1)
            GeoGradLine[startPtIdx_1:endPtIdx_1] = (m_slope_1*x+b_1).reshape(len(x),1)              
            sigma = np.random.randint(1,3)
            TemperatureCurves[m, :] = gaussian_filter(GeoGradLine[:,0], sigma=sigma)
            startPerturbPoints[m] = endPtIdx_1+MoveGaussian[sigma]
            if(e>=n_x-2):
                 EndPerturbPoints[m] = e
            else:
                EndPerturbPoints[m] = e-MoveGaussian[sigma]
        else:#situation number 3
            s = int(np.random.uniform(0.5*n_x, 0.85*n_x))
            e = n_x-1
            GeoGradLine[s:e+1] = GeoGradLine[s].copy()
            sigma = np.random.randint(1,15)
            TemperatureCurves[m, :] = gaussian_filter(GeoGradLine[:,0], sigma=sigma)
            startPerturbPoints[m] = s+MoveGaussian[sigma]
            EndPerturbPoints[m] = e
            
            
            
        ReservoirTemperature[m] = GeoGradLine[e].copy()

        PressureGradLine = np.linspace(startP, TargetPressures[m], num= n_x)
        PressureCurves[m,:] = PressureGradLine.copy().reshape((n_x,))
        midResPoint = int((startPerturbPoints[m]+EndPerturbPoints[m])/2)
        ReservoirPressure[m] = PressureGradLine[midResPoint] 
        ReservoirPressureDepth[m] = (startPerturbPoints[m]+EndPerturbPoints[m])/2
           
    Y = np.hstack((startPerturbPoints, EndPerturbPoints, ReservoirTemperature, ReservoirPressure, ReservoirPressureDepth))
    X = np.zeros((n_m, n_x, 2))
    X[:, :, 0] = TemperatureCurves
    X[:, :, 1] = PressureCurves
    np.savetxt('TrainingDataX1.csv', X[:, :, 0], delimiter=',')
    np.savetxt('TrainingDataX2.csv', X[:, :, 1], delimiter=',')
    np.savetxt('TrainingDataY.csv', Y, delimiter=',')

    return X, Y
  
print('congrats! you loaded the function into memory')

congrats! you loaded the function into memory


## 1.2 Call function to make synthetic data set

In [ ]:
# import plotting package
import matplotlib.pyplot as plt
# close any existing plots
plt.close("all")

# how many points discretization per log?
n_x=200
well_depth = 2000
dx = well_depth/n_x
print('Discretization is: ' +str(dx))

# how many training samples?
n_m= 30000

X, Y = CreateTrainingData5(n_m=n_m, n_x=n_x)
print('Data is ready!')

Discretization is: 10.0


# 2. Attribute engineering: add the derivative

In [ ]:
print('data shape before added new attribute: ' + str(np.shape(X)))
gradient = np.gradient(X[:, :, 0], axis=1) 
X = np.concatenate((X, gradient.reshape(n_m, n_x, 1)), axis=2)
print('data shape after added new attribute: ' + str(np.shape(X)))

## 1.3 plot the training data + new attribute --> derivative of the temperature log 

In [ ]:
# the number of samples to visualize
n_samples_viz = 20

# Depth points
Depth = np.arange(dx, dx*n_x+dx, dx)

for sampleN in range(n_samples_viz):

    f, axs = plt.subplots(nrows=1, ncols=3, sharey='row')

    # plot the temperature log
    axs[0].plot(X[sampleN, :,0], Depth, lw=1, alpha=1, color='r', label=r'Temperature log')
    # plot the temperature feed zones
    feedzone1Z = Y[sampleN, 0]
    axs[0].plot([0, 280], [dx*feedzone1Z,dx*feedzone1Z], lw=1, alpha=1, color='b', label=r'Feed zone 1')
    feedzone2Z = Y[sampleN, 1]
    axs[0].plot([0, 280], [dx*feedzone2Z,dx*feedzone2Z], lw=1, alpha=1, color='b', label=r'Feed zone 2')
    # plot text of the reservoir temperature
    ResTemperature = Y[sampleN, 2]
    axs[0].text(20, dx*feedzone2Z+100, 'Res Temp: ' + '{:.1f}'.format(ResTemperature), fontsize=12)
    axs[0].set_xlim([min(X[sampleN, :,0]), max(X[sampleN, :,0])+10])

    axs[0].set_xlabel(r'Temperature (C)')
    axs[0].set_ylabel('Depth')
    axs[0].set_title('Temperature')
    axs[0].legend(bbox_to_anchor=(-1.65, 1.02), loc="upper left")

    # plot the derivative ofthe temperature log    
    axs[1].plot(X[sampleN, :,2], Depth, lw=1, alpha=1, color='k', label=r'Temperature log')
    axs[1].set_title('Temp derivative')

    # plot the pressure log
    axs[2].plot(X[sampleN, :, 1], Depth, lw=1, alpha=1, color='r', label=r'Pressure log')
    PressureDepth = Y[sampleN, 4]*dx
    ResPressure = Y[sampleN, 3]
    axs[2].plot([0, 20], [PressureDepth,PressureDepth], lw=1, alpha=1, color='b', label=r'Pressure Depth Point')
    axs[2].set_xlabel(r'Pressure (MP)')
    axs[2].set_title('Pressure vs. depth')
    axs[2].invert_yaxis()
    axs[2].legend(bbox_to_anchor=(1.04, 1.02), loc="upper left")
    axs[2].text(10, PressureDepth-100, 'Res Pressure: ' + '{:.1f}'.format(ResPressure), fontsize=12)

## 1.4 Normalize the input data

In [ ]:
# Compute and save the current mean and standard deviation (std) values
Xmean = np.mean(X, axis=1)
Xstd = np.std(X, axis=1)  
np.savetxt('Xmean.csv', Xmean, delimiter=',')
np.savetxt('Xstd.csv', Xstd, delimiter=',')  

# Normalize the input data 
Xmean = np.reshape(Xmean, (n_m, 1, 3))
Xstd = np.reshape(Xstd, (n_m, 1, 3))    
X = (X-Xmean)/Xstd
#RangeTemp = Xmax[:,:,0]-Xmin[:,:,0]
#RangePressure = Xmax[:,:,1]-Xmin[:,:,1]
#MinTemp = Xmin[:,:,0]
#MinPressure = Xmin[:,:,1]
print('done loading and normalizing data')

# 2. Split the data into training, validation, and testing 

In [ ]:
ratio_train = 0.8
ratio_val = 0.1
ratio_test = 0.1
n_train = int(ratio_train*n_m)
n_val = int(ratio_val*n_m)
n_test = n_m-n_val-n_train

print(str(n_train) + ' samples in training')
print(str(n_val) + ' samples in validation')
print(str(n_test) + ' samples in testing')

XTrain = X[0:n_train]
YTrain = Y[0:n_train]

XVal = X[n_train:n_train+n_val]
YVal = Y[n_train:n_train+n_val]

XTest = X[n_train+n_val:]
YTest = Y[n_train+n_val:]

#3. Train model

## 3.1 load lots of packages

In [ ]:
# import machine learning models
import os
import multiprocessing
from multiprocessing import Lock, Process, Queue, current_process
import time
import queue # imported for using queue.Empty exception
import multiprocessing
import numpy as np
import os
import csv
import random   

import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import Model, Sequential
from keras.layers import Input
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.layers import BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.layers import Activation
from keras.layers import Conv1D, MaxPooling1D, Dense
from keras.layers import Dropout, Flatten
from keras.utils import np_utils
from tensorflow.python.framework import ops
import math
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from keras.models import load_model

%load_ext tensorboard
from tensorboard.plugins.hparams import api as hp
run_n = 0
print('finished loading packages')

## 3.2 load overarching architecture function

In [ ]:
# =============================================================================
# This section is where the CNN model is defined
# =============================================================================
def conv_block_first(model, params):
    """
    The first convolutional block in each architecture. 
    Only separate from the other blocks so we can specify the input shape.
    """    

    #First Stacked Convolution
    model.add(Conv1D(filters = params['Conv1D_1_output_filters'], 
                     kernel_size = params['Conv1D_1_kernel_size'], 
                     padding = params['Conv1D_1_padding'], 
                     input_shape = params["input_shape"]))

    if params['Conv1D_1_batchnorm']:
        model.add(BatchNormalization())
        
    model.add(Activation(params['Conv1D_1_activation']))
    
    if params['Conv1D_1_maxpool']:
        model.add(MaxPooling1D(pool_size=params['Conv1D_1_maxpool_size'], 
                               padding = params['Conv1D_1_maxpool_padding'], 
                               data_format='channels_last'))

    if params['Conv1D_1_dropout']:
        model.add(Dropout(params['Conv1D_1_dropout_rate']))
    return model

def conv_block(model, params):
    """
    Generic convolutional block, including:
      - convolutions, 
      - max pooling, 
      - dropout, 
      - and an optional Batch Normalization.
    """
    model.add(Conv1D(filters = params['Conv1D_mid_output_filters'], 
                     kernel_size = params['Conv1D_mid_kernel_size'], 
                     padding = params['Conv1D_mid_padding']))
    
    if params['Conv1D_mid_batchnorm']:
        model.add(BatchNormalization())
    
    model.add(Activation(params['Conv1D_mid_activation']))

    if params['Conv1D_mid_maxpool']:      
        model.add(MaxPooling1D(pool_size=params['Conv1D_mid_maxpool_size'],
                               padding = params['Conv1D_mid_maxpool_padding'],
                               data_format='channels_last'))
    
    if params['Conv1D_mid_dropout']: 
        model.add(Dropout(params['Conv1D_mid_dropout_rate']))
    
    return model

def fn_block(model):
    """
    The final block.
    """
    model.add(Flatten())
    # we have here dense of 5 because we are expecting five outputs
    model.add(Dense(5))
    return model

def build_model(params):
    """
    Builds a sequential network based on the specified parameters.
    
    blocks: number of convolutional blocks in the network, must be greater than 2.
    """
    model = Sequential()

    model = conv_block_first(model, params)

    for block in range(1,params['num_conv_blocks']):
        model = conv_block(model, params)

    model = fn_block(model)

    return model

## 3.3 Set the hyper parameters

In [ ]:
#set the ranges for the hyper-parameter space
start = time.time()
params= {'Conv1D_1_batchnorm': 'True', 'Conv1D_1_activation':'relu', 'Conv1D_1_maxpool': 'True', 'Conv1D_1_maxpool_padding': 'same', 
         'Conv1D_1_dropout': 'False', 'Conv1D_mid_padding':'same', 'Conv1D_mid_batchnorm':'True', 'Conv1D_mid_maxpool_padding': 'same',
         'Conv1D_mid_dropout': 'True', 'Conv1D_mid_maxpool':'True', 'Conv1D_1_dropout_rate': 0.1, 'Conv1D_mid_dropout_rate': 0.1, 
         'decay': 0.0001, 'Conv1D_1_output_filters': 19, 'batch_size': 100, 'batch_size': 100, 'Conv1D_mid_kernel_size': 3}

##########################
# Discrete parameters
# divided into parameters for the first block, middle blocks, and final block
##########################

# convolution filter edge treatment for first convolution block
# 'Conv1D_1_padding':['valid', 'same']
params['Conv1D_1_padding'] = 'valid'; 

# The activation used for the first convolution block
# 'Conv1D_mid_activation': ['relu', 'sigmoid', 'tanh']
params['Conv1D_mid_activation'] = 'relu'

###############################
# float values
##############################

# 'learning_rate':[0.001, 0.007]
params['learning_rate'] = 0.002

###############################
# integer parameters
# shown with the min and max recommended values
##############################

# The number of covolution blocks
# 'num_conv_blocks': [2, 7]
params['num_conv_blocks'] = 3

# The kernel/filter size for the first convolutional block
# 'Conv1D_1_kernel_size':[3, 30] 
params['Conv1D_1_kernel_size'] = 7

# The number of filters in the middle convolution blocks
# 'Conv1D_mid_output_filters': [3, 35]
params['Conv1D_mid_output_filters'] = 16

# the size of the kernel in the first maxpooling
# 'Conv1D_1_maxpool_size':[2, 4]
params['Conv1D_1_maxpool_size'] = 3

# the size of the kernel in the middle blocks
# 'Conv1D_mid_maxpool_size':[2, 4]
params['Conv1D_mid_maxpool_size'] = 3

# The filter size in the middle convolution block
# 'Conv1D_mid_kernel_size': [3, 30]
params['Conv1D_mid_kernel_size'] = 3

print('you finished setting the hyper parameters')

## 3.4 create the CNN model!

In [ ]:
# determine the data input shape
input_shape = XTrain.shape[1:]
params["input_shape"] = input_shape

# build the model
model = build_model(params)

# create an optimizer object
optimizer = keras.optimizers.Adam(lr=params['learning_rate'],
                                  beta_1=0.9, 
                                  beta_2=0.999, 
                                  clipnorm=1, 
                                  decay=params['decay'])

# compile the model and set the tracking parameters
model.compile(optimizer = optimizer,
              loss = "mean_squared_error",
              metrics = [tf.keras.metrics.MeanAbsoluteError()])

# view the model summary
model.summary()

## 3.5 run the model! Loop over 4 options of values for the number of convolution blocks

In [ ]:
check_out_num_blocks = [0, 1, 2, 3]
histories = [0,0,0,0]
for i in range(4):

    params['num_conv_blocks'] = check_out_num_blocks[i]

    params['filename'] = 'run_'+str(i) + '_' + str(check_out_num_blocks[i])+'_blocks'

    model_checkpoint = ModelCheckpoint(params['filename']+'.h5', 
                                      monitor='val_mean_absolute_error',
                                    save_best_only = True)


    log_dir = 'tb_logs/'+params['filename']
    tb = TensorBoard(log_dir=log_dir)
    file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
    file_writer.set_as_default()

    hparams = {i:params[i] for i in params if i in ['Conv1D_mid_kernel_size', 'num_conv_blocks', 'learning_rate']}
    histories[i] = model.fit(x=XTrain, y=YTrain,
                        batch_size=params['batch_size'],
                        epochs=15, 
                        verbose=1, 
                        callbacks=[model_checkpoint, tb, hp.KerasCallback(log_dir, hparams, trial_id=params['filename'])], 
                        validation_data=(XVal, YVal), 
                        shuffle=True,
                        class_weight=None, sample_weight=None, initial_epoch=0)
    
    runName = str(params['filename']) + '.h5'
    new_model = load_model(runName)
    Prediction = new_model.predict(XVal)
    mean_abs_error = np.mean(np.abs(Prediction - YVal))

    tf.summary.scalar('validation_mae', data=mean_abs_error, step=0)


## 3.6 Plot the error decrease

In [ ]:
history = histories[0] 

f = plt.figure(figsize=(12,6))

plt.plot(history.history['mean_absolute_error'], label ='training mean abs err')
plt.plot(history.history['val_mean_absolute_error'], label='validation mean abs err')
plt.ylabel('mean_absolute_error')
plt.xlabel('epoch')
plt.legend()
f.savefig('val_mean_absolute_error.png',dpi=300,bbox_inches='tight')

## 3.7 plot with tensor board

In [ ]:
%tensorboard --'logdir' 'tb_logs'

# 4. Evaluate results on testing set

In [ ]:
runName = str(params['filename']) + '.h5'
new_model = load_model(runName)
Prediction = new_model.predict(XTest)

mean_abs_error = np.mean(np.abs(Prediction - YTest))
print('Testing mean absolute error is: ' + str(mean_abs_error))

#reset Truth and Pred Y
rangeDepth=10
minDepth=0

Depth = np.linspace(dx, n_x*dx, n_x)
alpha = 0.8
lw = 2
Colors = ['r', 'g', 'b', 'k', 'y']
Depth_i = np.linspace(0, 2000, n_x)
absError = np.zeros((12,1))

txt_x = -0.7
txt_y = 250

for n in range(20):
    f, axs = plt.subplots(1, 2, sharey='row')

    # plot temperature 
    axs[0].plot(XTest[n, :, 0], Depth_i, lw=lw, alpha=alpha, color='r', 
                label=r'Log')

    # plot feed zones - truth 
    feedzone1Z = dx*YTest[n, 0]
    axs[0].plot([np.min(XTest[n, :, 0]), np.max(XTest[n, :, 0])],
                [feedzone1Z,feedzone1Z], 
                lw=lw, alpha=alpha, color='g', label=r'Truth - Feed zone 1')
    feedzone2Z = dx*YTest[n, 1]
    axs[0].plot([np.min(XTest[n, :, 0]), np.max(XTest[n, :, 0])],
                [feedzone2Z,feedzone2Z], lw=lw, alpha=alpha, color='g',
                label=r'Truth - Feed zone 2')

    # plot feed zones - prediction 
    feedzone1Z_p = dx*Prediction[n, 0]
    axs[0].plot([np.min(XTest[n, :, 0]), np.max(XTest[n, :, 0])],
                [feedzone1Z_p,feedzone1Z_p], lw=lw, alpha=alpha, 
                color='b',linestyle='dashed', label=r'Preiction - Feed zone 1')
    feedzone2Z_p = dx*Prediction[n, 1]
    axs[0].plot([np.min(XTest[n, :, 0]), np.max(XTest[n, :, 0])],
                [feedzone2Z_p,feedzone2Z_p], lw=lw, alpha=alpha, 
                color='b',linestyle='dashed', label=r'Prediction - Feed zone 2')

    # plot reservoir temperature truth vs. prediction
    ResTempT = YTest[n, 2]
    ResTempP = Prediction[n, 2]
    axs[0].text(txt_x, txt_y, 
                'Temp True: ' + '{:.0f}'.format(ResTempT), fontsize=12)
    axs[0].text(txt_x, txt_y+160, 
                'Temp Pred: ' + '{:.0f}'.format(ResTempP), fontsize=12)
    axs[0].set_ylim([0, 2000])

    # plot pressure curve
    axs[1].plot(XTest[n, :, 1], Depth_i, lw=lw, alpha=alpha, color='r', label=r'Log')

    # plot reservoir pressure depth truth vs. prediction
    ResPressureDepth = dx*YTest[n, 4]
    axs[1].plot([np.min(XTest[n, :, 1]), np.max(XTest[n, :, 1])], 
                [ResPressureDepth,ResPressureDepth], lw=lw, alpha=alpha, 
                color='g', label=r'Truth')
    ResPressureDepthPred = dx*Prediction[n, 4]
    axs[1].plot([np.min(XTest[n, :, 1]), np.max(XTest[n, :, 0])], [ResPressureDepthPred,ResPressureDepthPred], lw=lw, alpha=alpha, color='b',linestyle='dashed', label=r'Prediction')

    # plot reservoir pressure - truth vs prediction
    ResPresT = YTest[n, 3]
    ResPresP = Prediction[n, 3]
    axs[1].text(txt_x, txt_y, 'Pres True: ' + '{:.1f}'.format(ResPresT), fontsize=12)
    axs[1].text(txt_x, txt_y+160, 'Pres Pred: ' + '{:.1f}'.format(ResPresP), fontsize=12)

    axs[0].invert_yaxis()
    plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")

    absErrorT =np.mean(np.abs(YTest[n, 0:3] - Prediction[n, 0:3]))
    absErrorP =np.mean(np.abs(YTest[n, 3:] - Prediction[n, 3:]))

    axs[0].set_xlabel(r'Temperature')
    axs[0].set_ylabel('Depth')
    axs[0].set_title('Temperature (mean err='+str(np.round(absErrorT,1))+')')
    axs[1].set_xlabel(r'Pressure')
    axs[1].set_title('Pressure (mean err='+str(np.round(absErrorP,1))+')')

# 4. Visualize trained weights

In [ ]:
for layer in model.layers:
	# check for convolutional layer
	if 'conv' not in layer.name:
		continue
	# get filter weights
	filters, biases = layer.get_weights()
	print(layer.name, filters.shape)

In [ ]:
# retrieve weights from the second hidden layer
filters, biases = model.layers[0].get_weights()
# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)
# plot first few filters
n_filters, ix = 3, 1

fig, axs = plt.subplots(3, 3, figsize=(3,12))

for i in range(n_filters):
    # get the filter
    f = filters[:, :, i]
    # plot each channel separately
    for j in range(3):
        # plot filter channel in grayscale
        #ax = axs[i, j
        axs[i,j].imshow(f[:, j].reshape((-1,1)), cmap='gray')
        axs[i,j].axis('off')
        axs[i,j].set_xticks([])
        axs[i,j].set_yticks([])# show the figure

## 4.1 see the features after they were convolved with these kernels

In [ ]:
# redefine model to output right after the first hidden layer
model_1 = Model(inputs=model.inputs, outputs=model.layers[0].output)

In [ ]:
feature_maps = model_1.predict(XTrain[0].reshape(1,200,3))

In [ ]:
# retrieve weights from the second hidden layer
filters, biases = model.layers[0].get_weights()
# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

In [ ]:
np.linspace(0,2000, 200)[0]-np.linspace(0,2000, 200)[1]

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(4,18))

for i in range(3):
    axs[i, 0].plot(XTrain[0, :, 0], np.linspace(0,2000, 200))
    axs[i, 0].set_title('Temperature')
    axs[i, 0].set_ylim([0, 2000])
    axs[i, 0].invert_yaxis()

    axs[i, 1].step(filters[:,0,i], np.linspace(0,2000, 200)[0:len(filters[:,0,0])])
    axs[i, 1].set_title('filter 1,1')
    axs[i, 1].set_ylim([0, 200])
    axs[i, 1].invert_yaxis()

    axs[i, 2].plot(feature_maps[0, :, i], np.linspace(0,2000, len(feature_maps[0, :, i])))
    axs[i, 2].set_title('feature map')
    axs[i, 2].set_ylim([0, 2000])
    axs[i, 2].set_yticks([])
    axs[i, 2].invert_yaxis()


# One before last activity: a competition!

In [ ]:
run_n=0

## Step 1: play around with the hyper parameters and try to get the smallest validation error

In [ ]:
#set the ranges for the hyper-parameter space
start = time.time()
params= {'Conv1D_1_batchnorm': 'True', 'Conv1D_1_activation':'relu', 'Conv1D_1_maxpool': 'True', 'Conv1D_1_maxpool_padding': 'same', 
         'Conv1D_1_dropout': 'False', 'Conv1D_mid_padding':'same', 'Conv1D_mid_batchnorm':'True', 'Conv1D_mid_maxpool_padding': 'same',
         'Conv1D_mid_dropout': 'True', 'Conv1D_mid_maxpool':'True', 'Conv1D_1_dropout_rate': 0.1, 'Conv1D_mid_dropout_rate': 0.1, 
         'decay': 0.0001, 'Conv1D_1_output_filters': 19, 'batch_size': 100, 'batch_size': 100, 'Conv1D_mid_kernel_size': 3}

##########################
# Discrete parameters
# divided into parameters for the first block, middle blocks, and final block
##########################

# convolution filter edge treatment for first convolution block
# 'Conv1D_1_padding':['valid', 'same']
params['Conv1D_1_padding'] = 'valid'; 

# The activation used for the first convolution block
# 'Conv1D_mid_activation': ['relu', 'sigmoid', 'tanh']
params['Conv1D_mid_activation'] = 'relu'

###############################
# float values
##############################

# 'learning_rate':[0.001, 0.007]
params['learning_rate'] = 0.003

###############################
# integer parameters
# shown with the min and max recommended values
##############################

# The number of covolution blocks
# 'num_conv_blocks': [2, 7]
params['num_conv_blocks'] = 3

# The kernel/filter size for the first convolutional block
# 'Conv1D_1_kernel_size':[3, 30] 
params['Conv1D_1_kernel_size'] = 3

# The number of filters in the middle convolution blocks
# 'Conv1D_mid_output_filters': [3, 35]
params['Conv1D_mid_output_filters'] = 16

# the size of the kernel in the first maxpooling
# 'Conv1D_1_maxpool_size':[2, 4]
params['Conv1D_1_maxpool_size'] = 3

# the size of the kernel in the middle blocks
# 'Conv1D_mid_maxpool_size':[2, 4]
params['Conv1D_mid_maxpool_size'] = 3

# The filter size in the middle convolution block
# 'Conv1D_mid_kernel_size': [3, 30]
params['Conv1D_mid_kernel_size'] = 3

print('you finished setting the hyper parameters')

In [ ]:
# determine the data input shape
input_shape = XTrain.shape[1:]
params["input_shape"] = input_shape

# build the model
model = build_model(params)

# create an optimizer object
optimizer = keras.optimizers.Adam(lr=params['learning_rate'],
                                  beta_1=0.9, 
                                  beta_2=0.999, 
                                  clipnorm=1, 
                                  decay=params['decay'])

# compile the model and set the tracking parameters
model.compile(optimizer = optimizer,
              loss = "mean_squared_error",
              metrics = [tf.keras.metrics.MeanAbsoluteError()])

# view the model summary
model.summary()

run_n=run_n+1

params['filename'] = 'myrun_'+str(run_n)
    
model_checkpoint = ModelCheckpoint(params['filename']+'.h5', 
                                  monitor='val_mean_absolute_error',
                                save_best_only = True)

log_dir = 'tb_mylogs/myrun_'+str(run_n)
tb = TensorBoard(log_dir=log_dir)
file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
file_writer.set_as_default()


hparams = {i:params[i] for i in params if i in ['Conv1D_1_padding', 'Conv1D_mid_activation', 'learning_rate', 'Conv1D_1_kernel_size', 'Conv1D_mid_output_filters', 'Conv1D_1_maxpool_size', 'Conv1D_mid_maxpool_size', 'num_conv_blocks', 'Conv1D_mid_kernel_size']}
history = model.fit(x=XTrain, y=YTrain,
                    batch_size=params['batch_size'],
                    epochs=15, 
                    verbose=1, 
                    callbacks=[model_checkpoint, tb, hp.KerasCallback(log_dir, hparams, trial_id=params['filename'])], 
                    validation_data=(XVal, YVal), 
                    shuffle=True,
                    class_weight=None, sample_weight=None, initial_epoch=0)

## add the validation error to tensor board
runName = str(params['filename']) + '.h5'
new_model = load_model(runName)
Prediction = new_model.predict(XVal)
mean_abs_error = np.mean(np.abs(Prediction - YVal))
tf.summary.scalar('validation_mae', data=mean_abs_error, step=0)

In [ ]:
%tensorboard --'logdir' 'tb_mylogs'

### Check out the testing error of different runs

In [ ]:
num = run_n
runName = str('myrun_'+str(num)) + '.h5'
new_model = load_model(runName)
Prediction = new_model.predict(XTest)

mean_abs_error = np.mean(np.abs(Prediction - YTest))
print('Testing mean absolute error is: ' + str(mean_abs_error))

# ONLY GO INTO THIS SECTION WHEN INSTRUCTOR SAYS TO:

In [ ]:
# get an image from my github
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/ahinoamp/CNN_temp_log_tutorial/main/TestingDataX1.csv

!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/ahinoamp/CNN_temp_log_tutorial/main/TestingDataX2.csv

!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/ahinoamp/CNN_temp_log_tutorial/main/TestingDataY.csv

!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/ahinoamp/CNN_temp_log_tutorial/main/TestingDataX3.csv

UnseenTestingDataX = np.zeros((4000, 200, 3))
UnseenTestingDataX[:, :, 0] = np.genfromtxt('TestingDataX1.csv', delimiter=',')
UnseenTestingDataX[:, :, 1] = np.genfromtxt('TestingDataX2.csv', delimiter=',')
UnseenTestingDataX[:, :, 2] = np.genfromtxt('TestingDataX3.csv', delimiter=',')

CompetitionY = np.genfromtxt('TestingDataY.csv', delimiter=',')

In [ ]:
num_best_model = 1
modelname = 'myrun_'+str(num_best_model) + '.h5'
best_model = load_model(modelname)
Prediction = best_model.predict(UnseenTestingDataX)

mean_abs_error = np.mean(np.abs(Prediction - CompetitionY))
print('Your competition mean absolute error is: ' + str(np.round(mean_abs_error,1)).replace('.', 'pt'))